In [2]:
import numpy as np
import qiskit as qk
from qiskit import Aer
from qiskit.compiler import transpile, assemble
import time

from src.neuralnetwork import *
%load_ext autoreload
%autoreload 2

### Looping over Circuits vs. Assembled Circuits Benchmark

In [6]:
np.random.seed(42)

backend = Aer.get_backend("qasm_simulator")
backend_options = {"method": "automatic","max_parallel_shots":1,"max_parallel_experiments":0}

In [7]:
n_qubits = 5
repeats = 1000
shots = 10000

In [8]:
start = time.time()
for i in range(repeats):
    storage = qk.QuantumRegister(n_qubits)
    clas_reg = qk.ClassicalRegister(1)
    
    circuit = qk.QuantumCircuit(storage, clas_reg)
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])

    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])
        
    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
       
    circuit.measure(storage[-1], clas_reg)
    
    job = qk.execute(circuit, backend, shots=shots)
    result = job.result()

end = time.time()
print(end-start)

17.75918936729431


In [7]:
start = time.time()
circuit_list = []
for i in range(repeats):
    storage = qk.QuantumRegister(n_qubits)
    clas_reg = qk.ClassicalRegister(1)

    circuit = qk.QuantumCircuit(storage, clas_reg)
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])

    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])
        
    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
    
    circuit.measure(storage[-1], clas_reg)
    
    circuit_list.append(circuit) 
    

transpiled_list = transpile(circuit_list, backend=backend)
qobject_list = assemble(transpiled_list, backend=backend, shots=shots, max_parallel_shots = 1, max_parallel_experiments = 0)
job = backend.run(qobject_list)

result = job.result()

end = time.time()
print(end-start)

2.951052665710449


### Multiple samples feed-forward layer

In [40]:
x = np.array([[1,2,3], [4,5,6], [7,8,9], [10,11,12]])

In [41]:
np.random.seed(42)
layer = QLayer(n_qubits=3, 
               n_features=3, 
               n_targets=2, 
               encoder=Encoder(), 
               ansatz=Ansatz(), 
               reps=1, 
               scale=np.pi, 
               backend=backend, 
               shots=100000)

In [44]:
y = [layer(x_.reshape(1,-1)) for x_ in x]
print(y)

[array([[0.54220748, 1.40652245]]), array([[0.67751587, 1.22346184]]), array([[0.78194241, 1.05840256]]), array([[0.8233486 , 0.97389372]])]


In [42]:
y = layer(x)
print(y)

[[0.53938004 1.40476316]
 [0.68282516 1.22006892]
 [0.78335613 1.07065478]
 [0.81599728 0.98338133]]


### Multiple samples backward

In [82]:
x = np.array([[1,2,3], [4,5,6], [7,8,9], [10,11,12]], dtype="float64")
y = np.array([[1], [2], [3], [4]])

In [87]:
np.random.seed(42)
layer1 = QLayer(n_qubits=3, 
                n_features=3, 
                n_targets=2, 
                encoder=Encoder(), 
                ansatz=Ansatz(), 
                reps=1, 
                scale=np.pi, 
                backend=backend, 
                shots=100000)

layer2 = QLayer(n_qubits=2, 
                n_features=2, 
                n_targets=1, 
                encoder=Encoder(), 
                ansatz=Ansatz(), 
                reps=1, 
                scale=np.pi, 
                backend=backend, 
                shots=2000)
layer = [layer1, layer2]

optimizer = Adam()

network = NeuralNetwork(layer, optimizer)

In [88]:
network.backward(x,y)

In [89]:
print(network.weight_gradient_list)

[array([[-0.51898463,  0.49561193],
       [ 0.20798833,  0.13357685],
       [-0.03951589,  1.11504249]]), array([[-0.08531373],
       [ 0.60358019]])]


In [90]:
network.step()